# Moving Averages in Time-series Data

We can leverage the concept of [shift-based methods](./shift-methods.qmd) to calculate our own trends and moving averages in time-series data.

## Simple vs Exponential Weighted Moving Average

We will implement two different kinds of moving average:

  + Rolling Window Averages, using the [`rolling` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html)
  + Exponential Weighted Moving Averages, using the [`ewm` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.ewm.html)

Let's consider the following time series dataset of stock prices:


In [ ]:
from pandas import read_csv

request_url = "https://raw.githubusercontent.com/prof-rossetti/python-for-finance/main/docs/data/daily_adjusted_NFLX.csv"
prices_df = read_csv(request_url)
prices_df.head()

Because we'll be using shift-based methods, we must first sort the data, as usual, by date in ascending order:


In [ ]:
prices_df.sort_values(by=["timestamp"], inplace=True) #
prices_df.head()

In [ ]:
prices_df["ma_50"] = prices_df["adjusted_close"].rolling(window=50).mean()

prices_df["ema_50"] = prices_df["adjusted_close"].ewm(span=50, min_periods=0, adjust=False, ignore_na=False).mean()

prices_df[["timestamp", "adjusted_close", "ma_50", "ema_50"]]

In [ ]:
import plotly.express as px

px.line(prices_df, x="timestamp", y=["close", "ma_50", "ema_50"],
        title=f"Adjusted Closing Prices",
        color_discrete_map={
            "close": "royalblue",
            "ma_50": "orange",
            "ema_50":"yellow"
        }
)

You'll notice there are no values for the first N number of periods in our rolling window average (where N is the size of the window). This is because there aren't enough values to complete the average. It's OK!

If you would like to change this behavior to be less methodologically strict, we can apply the `min_periods` parameter, setting minimum number of periods to zero, in which case as many values will be used until we get to fifty, at which point the true rolling average takes over:


In [ ]:
# SETTING MIN PERIODS = ZERO:
prices_df["ma_50_min_0"] = prices_df["adjusted_close"].rolling(window=50, min_periods=0).mean()

px.line(prices_df, x="timestamp",
        y=["close", "ma_50_min_0", "ma_50", "ema_50"],
        title=f"Adjusted Closing Prices",
        color_discrete_map={
            "close": "royalblue",
            "ma_50_min_0": "pink",
            "ma_50": "orange",
            "ema_50":"yellow"
        }
)

## Golden Cross

<img src="https://www.investopedia.com/thmb/2ZfXiCx5edgO3ITq8pYmdphrAGQ=/1500x0/filters:no_upscale():max_bytes(150000):strip_icc()/goldencross-1323afb7bf1f412a9bdf0b058710d6e5.jpg" height=350>

Reference: <https://www.investopedia.com/terms/g/goldencross.asp>

> "A Golden Cross is a chart pattern in which a relatively short-term moving average crosses above a long-term moving average. It is a bullish breakout pattern that forms when a security's short-term moving average (such as the 50-day moving average) crosses above its long-term moving average (such as the 200-day moving average) or resistance level."

For this "golden cross" analysis we will use dataset of Bitcoin prices:


In [ ]:
from pandas import read_csv, to_datetime

def fetch_crypto_data():
    request_url = (
        "https://raw.githubusercontent.com/"
        "prof-rossetti/applied-data-science-python-book/"
        "main/docs/data/currency_daily_btc_usd_20141123.csv"
    )
    df = read_csv(request_url)
    df.index = to_datetime(df["timestamp"])
    df.drop(columns=["timestamp"], inplace=True)
    return df

df = fetch_crypto_data()
df.head()

:::{.callout-note title="Data Source"}
Recent historical prices for Bitcoin, from the AlphaVantage API, accessed on November 23, 2024.

Source: <https://www.alphavantage.co/documentation/#currency-daily>
:::


Calculating the short-term and long-term moving averages, using 50 and 200 days respectively:


In [ ]:
def add_moving_avg(df, window=50):
    df = df.sort_index() # assumes index is datetime-aware
    df[f"MA_{window}"] = df["close"].rolling(window=window).mean()
    return df

df = add_moving_avg(df, window=50)
df = add_moving_avg(df, window=200)

df = df.dropna()
df = df.round(2)
df.head()

Identifying "golden cross" points where the 50-day average surpasses the 200-day average:


In [ ]:
has_crossed = (df['MA_50'] > df['MA_200'])
not_previously_crossed = (df['MA_50'].shift(1) <= df['MA_200'].shift(1))
df['Golden_Cross'] = (has_crossed & not_previously_crossed)

golden_cross_points = df[df['Golden_Cross'] == True]
golden_cross_points[["close", "MA_50", "MA_200"]].head()

Visualizing the golden cross points:


In [ ]:
title = "Bitcoin Prices with Moving Average and Golden Cross"
colors_map = {"close": "steelblue", "MA_50": "yellow", "MA_200": "orange"}

fig = px.line(df, y=["close", "MA_50", "MA_200"], height=450, title=title,
    color_discrete_map=colors_map
)

fig.add_scatter(
    x=golden_cross_points.index,
    y=golden_cross_points['MA_50'],
    mode='markers',
    marker=dict(size=12, color='yellow', symbol='x'),
    name='Golden Cross'
)
fig.show()

## Bollinger Bands

<img src="https://www.investopedia.com/thmb/W6lsIWt9KIrGcvOndRHcOc-AKgY=/750x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/bollinger-band-4200502-06f2a74c1c9d471085cf8c2f3a4fc514.jpg" height=350>

Reference: <https://www.investopedia.com/terms/b/bollingerbands.asp>

> "Bollinger Bands is a technical analysis tool developed by John Bollinger in the 1980s to help investors and traders gauge market volatility and identify when securities are poised to rise or fall."

> "The three lines that make up Bollinger Bands are based on a security's price moves. The center line is the intermediate-term trend and is typically a 20-day SMA of the closing prices. The upper and lower bands are plotted a distance from the SMA set by a certain number of standard deviations, usually two, above and below the center line."

Loading the data and sorting time series by date in ascending order, as necessary:


In [ ]:
df = fetch_crypto_data()
df = df.sort_index() # assumes index is datetime-aware

Calculating moving average, moving standard deviation, and upper and lower bands:


In [ ]:
window = 50
ma_col = f"MA_{window}"
std_col = f"STD_{window}"
df[ma_col] = df["close"].rolling(window=window).mean()
df[std_col] = df["close"].rolling(window=window).std()

band_std = 2
lower_col = f"LOWER_{window}"
upper_col = f"UPPER_{window}"
df[upper_col] = df[ma_col] + (band_std * df[std_col])
df[lower_col] = df[ma_col] - (band_std * df[std_col])

df = df.dropna()
df = df.round(2)

df[["close", ma_col, lower_col, upper_col]].head()

Plotting the bands:


In [ ]:
import plotly.express as px

title = "Bitcoin Price with Moving Average and Bollinger Bands"
colors_map = {"close": "steelblue", ma_col: "orange",
                upper_col: "yellow", lower_col: "yellow",
}
fig = px.line(df, y=["close", ma_col, upper_col, lower_col],
                title=title, color_discrete_map=colors_map,
                labels={"value": "Price (USD)", "timestamp": "Date"},
)
fig.update_layout(legend_title_text="Legend")
fig.show()

Plotting bands (alternative fill variant):


In [ ]:
from pandas import concat, Series

# fill regions:
fig.add_traces([
    dict(
        x=concat([Series(df.index), Series(df.index)[::-1]]),
        y=concat([df[upper_col], df[lower_col][::-1]]),
        fill="toself",
        #fillcolor="rgba(128, 0, 128, 0.2)",  # Light purple fill
        fillcolor="rgba(255, 255, 102, 0.2)",  # Light yellow fill
        line=dict(color="rgba(255,255,255,0)"),
        hoverinfo="skip",
        showlegend=False
    )
])
fig.show()

How can we interpret the chart? Are the bands getting narrower or wider?